In [27]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm, norm

In [29]:
def is_unitary(m):
    return norm(np.eye(len(m)) - m.dot(m.T.conj()))
def is_hermitian(m):
    return norm(m-m.T.conj())

In [52]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])

hamiltonian = np.zeros((2**10, 2**10), dtype=np.cdouble)
# comment this out when stuff actually works
coefficients = [0.25]*1
bases = ['XIIIIIIIII']# 'XXXXIIIIII', 'XXIIIIIIII', 'XXXIIIIIII'] # these are all commuting
# numTerms = len(bases)
for c, lbl in zip(coefficients, bases):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

t = 1
e_ith = expm(-1.j * t * hamiltonian)

In [63]:
e_ith
e_ith_new = expm(-1.j * t * 4 * hamiltonian)
e_ith_new_x = LA.matrix_power(e_ith, 4)

In [66]:
e_ith_new.shape

(1024, 1024)

In [67]:
norm(e_ith_new_x-e_ith_new)

6.153480596427404e-15

In [61]:
np.unique(hamiltonian)

array([0.  +0.j, 0.25+0.j])

In [53]:
is_hermitian(hamiltonian), is_unitary(e_ith)

(0.0, 3.66205343881779e-15)

In [92]:

n = 1 # number of time divisions. n=1 works for commuting operators 
qc = QuantumCircuit(10)
for i in range(n):
    for c, b in zip(coefficients, bases):
        # implement e^{-i*c*b*t/n}
        print(c, b)
        q = []
        qc.barrier()
        for i, op in enumerate(b):
            if (op == 'X'):
                qc.h(i)
                q.append(i)
            elif (op == 'Z'):
                q.append(i)
            elif (op == 'Y'):
                qc.h(i)
                qc.s(i)
                q.append(i)
            elif (op == 'I'):
                continue
        if (len(q) > 0):
            if (len(q) > 1):
                for i in range(len(q)-1):
                    qc.cx(q[i], q[-1])
            qc.rz(2*c*t/n, q[-1])
            if (len(q) > 1):
                for i in reversed(range(len(q)-1)):
                    qc.cx(q[i], q[-1])
            for i, op in enumerate(b):
                if (op == 'X'):
                    qc.h(i)
                elif (op == 'Z'):
                    continue
                elif (op == 'Y'):
                    qc.sdg(i)
                    qc.h(i)
                elif (op == 'I'):
                    continue
        qc.barrier()

0.25 XIIIIIIIII


In [93]:
qc.draw()

░ ┌───┐┌─────────┐┌───┐ ░ 
q_0: ─░─┤ H ├┤ Rz(0.5) ├┤ H ├─░─
      ░ └───┘└─────────┘└───┘ ░ 
q_1: ─░───────────────────────░─
      ░                       ░ 
q_2: ─░───────────────────────░─
      ░                       ░ 
q_3: ─░───────────────────────░─
      ░                       ░ 
q_4: ─░───────────────────────░─
      ░                       ░ 
q_5: ─░───────────────────────░─
      ░                       ░ 
q_6: ─░───────────────────────░─
      ░                       ░ 
q_7: ─░───────────────────────░─
      ░                       ░ 
q_8: ─░───────────────────────░─
      ░                       ░ 
q_9: ─░───────────────────────░─
      ░                       ░

In [85]:
backend = Aer.get_backend('unitary_simulator')
job = execute(qc, backend)
result = job.result()
U_circuit_4 = result.get_unitary(qc) 

In [87]:
norm(U_circuit_4-LA.matrix_power(U_circuit, 4))

3.552713678800501e-15

In [82]:
U_circuit

Operator([[ 9.68912422e-01+0.j        , -1.51491233e-17-0.24740396j,
            0.00000000e+00+0.j        , ...,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
          [ 1.51491233e-17-0.24740396j,  9.68912422e-01+0.j        ,
            0.00000000e+00+0.j        , ...,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
          [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
            9.68912422e-01+0.j        , ...,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
          ...,
          [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        , ...,  9.68912422e-01+0.j        ,
            0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
          [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        , ...,  0.00000000e+00+0.j

In [88]:
diff = U_circuit - e_ith
# measure the operator's accuracy
data = diff.data
w, v = LA.eig(data)

# measure the error relative to a matrix of 1s for reference.
ones = np.ones(data.shape)
fakeDiff = U_circuit - ones
w_fake, v_fake = LA.eig(fakeDiff.data)

In [91]:
diff = U_circuit_4 - e_ith_new
# measure the operator's accuracy
data = diff.data
w, v = LA.eig(data)
print(np.absolute(max(w)))

1.1916253585419323e-15


In [90]:
U_circuit

Operator([[ 9.68912422e-01+0.j        , -1.51491233e-17-0.24740396j,
            0.00000000e+00+0.j        , ...,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
          [ 1.51491233e-17-0.24740396j,  9.68912422e-01+0.j        ,
            0.00000000e+00+0.j        , ...,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
          [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
            9.68912422e-01+0.j        , ...,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
          ...,
          [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        , ...,  9.68912422e-01+0.j        ,
            0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
          [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
            0.00000000e+00+0.j        , ...,  0.00000000e+00+0.j

In [89]:
print(np.absolute(max(w)))

0.49480791850904576


10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)